In [0]:
import h5py

# The DigitStructFile is just a wrapper around the h5py data.  It basically references
#    input:              The input h5 matfile
#    digitStruct_name:   The h5 ref to all the file names
#    digitStruct_bbox:   The h5 ref to all struc data

class DigitStructFile:
    def __init__(self, input):
        self.input = h5py.File(input, 'r')
        self.digitStruct_name = self.input['digitStruct']['name']
        self.digitStruct_bbox = self.input['digitStruct']['bbox']

    # get_name returns the 'name' string for the n(th) digitStruct.

    def get_name(self,n):
        return ''.join([chr(c[0]) for c in self.input[self.digitStruct_name[n][0]].value])

    def bbox_helper(self,attribute):
        if (len(attribute) > 1):
            attribute = [self.input[attribute.value[j].item()].value[0][0] for j in range(len(attribute))]
        else:
            attribute = [attribute.value[0][0]]
        return attribute

    # get_bbox returns a 'dict' of data for the n(th) bbox.
    def get_bbox(self,n):
        bbox = {}
        bb = self.digitStruct_bbox[n].item()
        bbox['height'] = self.bbox_helper(self.input[bb]["height"])
        bbox['label']  = self.bbox_helper(self.input[bb]["label"])
        bbox['left']   = self.bbox_helper(self.input[bb]["left"])
        bbox['top']    = self.bbox_helper(self.input[bb]["top"])
        bbox['width']  = self.bbox_helper(self.input[bb]["width"])
        return bbox

    def get_DigitStructure(self,n):
        s = self.get_bbox(n)
        s['name']=self.get_name(n)
        return s

    # get_all_DigitStructure returns all the digitStruct from the input file.
    def get_all_DigitStructure(self):
        return [self.get_DigitStructure(i) for i in range(len(self.digitStruct_name))]
        #return [self.get_DigitStructure(i) for i in range(100)]

# Return a restructured version of the dataset (one structure by boxed digit).
#   Return a list of such dicts :
#      'filename' : filename of the samples
#      'boxes' : list of such dicts (one by digit) :
#          'label' : 1 to 9 corresponding digits. 10 for digit '0' in image.
#          'left', 'top' : position of bounding box
#          'width', 'height' : dimension of bounding box
    def get_all_DigitStructure_ByDigit(self):
        img_data = self.get_all_DigitStructure()
        result = []
        struct_count = 1
        for i in range(len(img_data)):
            item = { 'filename' : img_data[i]["name"] }
            figures = []
            for j in range(len(img_data[i]['height'])):
                figure = {}
                figure['height'] = img_data[i]['height'][j]
                figure['label']  = img_data[i]['label'][j]
                figure['left']   = img_data[i]['left'][j]
                figure['top']    = img_data[i]['top'][j]
                figure['width']  = img_data[i]['width'][j]
                figures.append(figure)
            struct_count += 1
            item['boxes'] = figures
            result.append(item)
        return result
